# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

## Norvig's solution 

In [8]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

file = open('fivegrams.txt', encoding='latin-1').read()
WORDS = Counter(words(file))

In [9]:
class NorvigSpellCorrector:
    def __init__(self):
        self.letters = 'abcdefghijklmnopqrstuvwxyz'
        self.WORDS = WORDS
    
    def probability(self, word, N=None):
        """Calculate the probability of `word`."""
        if N is None:
            N = sum(self.WORDS.values())
        return self.WORDS[word] / N
    
    def known(self, words):
        """The subset of `words` that appear in the dictionary of WORDS."""
        return set(w for w in words if w in self.WORDS)
    
    def edits1(self, word):
        "All edits that are one edit away from `word`."
        splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
        deletes    = [L + R[1:]               for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
        replaces   = [L + c + R[1:]           for L, R in splits if R for c in self.letters]
        inserts    = [L + c + R               for L, R in splits for c in self.letters]
        return set(deletes + transposes + replaces + inserts)
    
    def edits2(self, word):
        """All edits that are two edits away from `word`."""
        return (e2 for e1 in self.edits1(word) for e2 in self.edits1(e1))

    def candidates(self, word):
        """Generate possible spelling corrections for word."""
        return (self.known([word]) | self.known(self.edits1(word)) | self.known(self.edits2(word)) | {word})

    def correction(self, word):
        """Most probable spelling correction for word."""
        return max(self.candidates(word), key=self.probability)

    def correct_sentence(self, sentence):
        """Corrects all words within a sentence."""
        return ' '.join(self.correction(word) for word in re.findall(r'\w+', sentence.lower()))


# My solution

In [10]:
from collections import defaultdict

def update_ngram_counts(ngram_dict, ngram_tuple, increment):
    """
    Updates the count of an n-gram in a dictionary.

    Parameters:
    - ngram_dict (dict): Dictionary of n-gram counts.
    - ngram_tuple (tuple): The n-gram tuple to update.
    - increment (int): The value to add to the n-gram's count.

    Raises:
    - ValueError: If attempting to add a duplicate n-gram with no increment.
    """
    if ngram_dict[ngram_tuple] and increment == 0:
        raise ValueError(f"Duplicate detected for n-gram: {ngram_tuple}")
    ngram_dict[ngram_tuple] += increment

def extract_ngrams_and_update(line, gram5_dict, gram4_dict, gram3_dict, is_five_gram=True):
    """
    Extracts n-grams from a line of text and updates corresponding n-gram dictionaries.

    Parameters:
    - line (str): A line of text containing an n-gram and its count.
    - gram5_dict, gram4_dict, gram3_dict (dict): Dictionaries for 5-gram, 4-gram, and 3-gram counts.
    - is_five_gram (bool): True if processing a 5-gram, False if processing a 2-gram.

    Processes the line to extract the n-gram and its count, updates the 5-gram dictionary if applicable,
    and also updates 4-gram and 3-gram dictionaries based on the extracted n-gram.
    """
    parts = line.strip().split("\t")
    base_ngram = tuple(parts[1:])
    count = int(parts[0])

    if is_five_gram:
        update_ngram_counts(gram5_dict, base_ngram, count if gram5_dict[base_ngram] == 0 else 0)
        sub_ngrams = [base_ngram[1:], base_ngram[:-1]]  # 4-grams from 5-gram
    else:
        sub_ngrams = [base_ngram]

    # Update 4-gram and 3-gram dictionaries based on extracted n-grams
    for ngram in sub_ngrams:
        if len(ngram) == 4:
            update_ngram_counts(gram4_dict, ngram, count)
            # Generate and update 3-grams from 4-grams
            for i in range(1, 4):
                update_ngram_counts(gram3_dict, ngram[i-1:i+2], count)
        elif len(ngram) == 2 and not is_five_gram:
            update_ngram_counts(gram2_dict, ngram, count)

gram5_dict, gram4_dict, gram3_dict, gram2_dict = [defaultdict(int) for _ in range(4)]

# Process 5-gram file
with open('fivegrams.txt', 'r', encoding='latin-1') as file:
    [extract_ngrams_and_update(line, gram5_dict, gram4_dict, gram3_dict) for line in file]

# Process 2-gram file
with open('bigrams.txt', 'r', encoding='latin-1') as file:
    [extract_ngrams_and_update(line, gram2_dict, gram4_dict, gram3_dict, is_five_gram=False) for line in file]


In [11]:
def print_top_ngrams(ngram_dict, ngram_name, top_n=5):
    """Print the top N most common n-grams and their counts."""
    top_ngrams = sorted(ngram_dict.items(), key=lambda x: x[1], reverse=True)[:top_n]
    print(f"------- {ngram_name}: -------")
    print(f"Top {top_n} most common {ngram_name.lower()}:")
    for ngram, count in top_ngrams:
        print(f"{' '.join(ngram)}: {count}")
    print() 


print_top_ngrams(gram5_dict, "Fivegrams")
print_top_ngrams(gram4_dict, "Fourgrams")
print_top_ngrams(gram3_dict, "Threegrams")
print_top_ngrams(gram2_dict, "Twograms")

------- Fivegrams: -------
Top 5 most common fivegrams:
at the end of the: 13588
i do n't want to: 12744
in the middle of the: 9124
i do n't know what: 8076
you do n't have to: 7186

------- Fourgrams: -------
Top 5 most common fourgrams:
the end of the: 53263
do n't want to: 50903
i do n't think: 50304
i do n't know: 42473
in the united states: 39050

------- Threegrams: -------
Top 5 most common threegrams:
of the: 1253466
in the: 372495
to be: 306868
to the: 243312
i do n't: 230390

------- Twograms: -------
Top 5 most common twograms:
of the: 2586813
in the: 2043262
to the: 1055301
on the: 920079
and the: 737714



In [12]:
class ContextSearcher:
    def __init__(self, ngram_maps):
        """
        Initializes the ContextSearcher with a dictionary of n-gram maps.

        Parameters:
        - ngram_maps (dict): A dictionary where keys are integers indicating the n-gram size (e.g., 2 for bigrams, 3 for trigrams, etc.) 
                             and values are dictionaries mapping n-gram tuples to their occurrence counts in a corpus.
        """
        self.ngram_maps = ngram_maps 

    def get_context_score(self, word_sequence):
        """
        Retrieves the occurrence count (score) of a specific word sequence from the n-gram maps.

        Parameters:
        - word_sequence (tuple): A tuple of words representing the n-gram whose score is to be retrieved.

        Returns:
        int: The occurrence count of the given word sequence in the corpus. Returns 0 if the sequence is not found.
        """
        ngram_level = len(word_sequence)
        if ngram_level in self.ngram_maps:
            return self.ngram_maps[ngram_level].get(word_sequence, 0)
        return 0

    def find_contexts(self, target, precursors):
        """
        Identifies and scores contexts in which a target word appears, based on the precursor words provided.

        Parameters:
        - target (str): The target word for which contexts are being searched.
        - precursors (list): A list of words preceding the target word, used to construct n-grams for context search.
                             The list may contain more than 4 words, but only the last 4 are considered for up to a 5-gram context.

        Returns:
        list: A list of tuples, where each tuple contains two elements:
              1. The length of the context (n-1, where n is the n-gram size),
              2. The occurrence count (score) of the context in the corpus.
              The list is sorted by context length, with longer contexts appearing first.
        """
        precursors = precursors[-4:]  # Use up to 4 preceding words for a 5-gram context
        context_results = []

        for i in range(1, len(precursors) + 1):
            # Construct the sequence to check, ensuring it ends with the target word
            word_seq = tuple(precursors[-i:] + [target])
            score = self.get_context_score(word_seq)
            if score > 0:
                context_results.append((len(word_seq) - 1, score))

        # If no context found, attempt to get score for just the target word
        if not context_results:
            score = self.get_context_score((target,))
            if score > 0:
                context_results.append((1, score))

        return context_results

In [15]:
class WordRefiner:
    def __init__(self, context_searcher):
        """
        Initializes the WordRefiner with a context searcher for context-based spelling correction.

        Parameters:
        - context_searcher (ContextSearcher): An instance of the ContextSearcher class used to find contexts
          and scores for potential corrections.
        """
        self.searcher = context_searcher
        self.WORDS = WORDS  
    
    def P(self, word, N=None):
        """Probability of `word`."""
        if N is None:
            N = sum(self.WORDS.values())
        return self.WORDS[word] / N
    
    def known(self, words):
        """The subset of `words` that appear in the dictionary of WORDS."""
        return set(w for w in words if w in self.WORDS)

    def edits1(self, word):
        """All edits that are one edit away from `word`."""
        letters = 'abcdefghijklmnopqrstuvwxyz'
        splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes = [L + R[1:] for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
        replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
        inserts = [L + c + R for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)
    
    def candidates(self, word):
        """Generate possible spelling corrections for word."""
        return self.known([word]) | self.known(self.edits1(word)) | {word}

    def refine_word(self, incorrect_word, context_words):
        """
        Refines an incorrect word using the surrounding context for the best fit.

        Parameters:
        - incorrect_word (str): The word to refine.
        - context_words (list): A list of words preceding the word to refine, providing context.

        Returns:
        str: The most likely correction of the incorrect word based on context.
        """
        candidate_words = list(self.candidates(incorrect_word))
        context_scores = [self.searcher.find_contexts(cand, context_words) for cand in candidate_words]

        best_scores = [sorted(scores, key=lambda x: (-x[0], -x[1]))[0] if scores else (0, 0) for scores in context_scores]
        best_candidate_index = best_scores.index(max(best_scores, key=lambda x: (x[0], x[1])))

        return candidate_words[best_candidate_index]
    
    def correct_sentence(self, sentence):
        """
        Corrects all words within a sentence based on their context, refining each word individually.

        Parameters:
        - sentence (str): The sentence to correct.

        Returns:
        str: The corrected sentence, with each word refined based on context.
        """
        words = sentence.split()  # Split sentence into words
        corrected_words = []

        for i in range(len(words)):
            # Use preceding words as context for the current word
            context = corrected_words[-4:] if i >= 4 else corrected_words
            corrected_word = self.refine_word(words[i], context)
            corrected_words.append(corrected_word)
        
        return ' '.join(corrected_words)


In [16]:
ngram_maps = {2: gram2_dict, 3: gram3_dict, 4: gram4_dict, 5: gram5_dict}

spell_corrector = NorvigSpellCorrector()
context_searcher = ContextSearcher(ngram_maps)
word_refiner = WordRefiner(context_searcher)

In [17]:
sentence = "spellng errurs sometims vary bas"

print("Original sentence:", sentence)
print()
print("Corrected sentence by Norvings corrector:", spell_corrector.correct_sentence(sentence))
print("Corrected sentence by Context Sensetive corrector:", word_refiner.correct_sentence(sentence))

Original sentence: spellng errurs sometims vary bas

Corrected sentence by Norvings corrector: selling error sometimes are a
Corrected sentence by Context Sensetive corrector: spelling errors sometimes very bad


## One more example...

In [18]:
word = 'nic'
context_sent = 'Her smile is very'

print("Context:", context_sent)
print("Corrected word by Norvings corrector:", spell_corrector.correction(word))
print("Corrected word by Context Sensetive corrector:", word_refiner.refine_word(word, words(context_sent)))

Context: Her smile is very
Corrected word by Norvings corrector: in
Corrected word by Context Sensetive corrector: nice


# Justification

## Contextual Awareness

#### Key Improvement
Incorporation of context for choosing corrections.

#### Justification
While Norvig's model efficiently identifies potential corrections based on edit distance, it doesn't account for the contextual appropriateness of a word. By integrating n-grams up to 5-grams, my system assesses and selects corrections that align naturally within the sentence's broader context, significantly enhancing correction accuracy for contextually misplaced words.

## Dynamic Data Utilization

#### Key Improvement
Dynamic generation and use of 3 and 4-gram data from 5-grams.

#### Justification
Norvig's approach relies on a fixed dataset for correction suggestions, focusing on individual word probabilities and edit distances. My method dynamically slices 5-grams to generate 3 and 4-grams, providing a more comprehensive understanding of language patterns without needing separately maintained datasets. This improves performance and scalability.

## Enhanced Decision Making

#### Key Improvement
Prioritization of corrections based on context length and frequency.

#### Justification
My solution evaluates candidate words based on their context length and occurrence, ensuring that chosen corrections are not only probable but also contextually coherent. This addresses the issue of correct words feeling "out of place," a limitation in Norvig's simpler probability-based selection.

## Flexible Correction Approach

#### Key Improvement
Correction suggestions for known words based on edit distance and context.

#### Justification
Norvig's corrector accepts a known word as is, without considering potential contextual misfits. MY approach generates one-edit distance candidates even for known words, introducing flexibility and enabling the system to suggest alternatives that might better fit the context.

# Evaluate on a test set

In [ ]:
from tqdm import tqdm

def correct_text(corrector, text):
    """Corrects a sentence text using the specified corrector."""
    return corrector.correct_sentence(text)


## Evaluation on highly auhmented test set

In [59]:
import pandas as pd 

test_df = pd.read_csv('test.csv')
test_df = test_df.sample(500)
test_df.head()

,text,augmented_text
3760,vic armed offenders squad above the law,vic arN$d offenders s!uaw abpvs the law
34462,gold coast police rescues a man and woman off ...,gold coast ploiec rsceues a man and woman off ...
100482,high end rental market slumps as mining boom t...,hUisgh end rental market sOluEmps as mining bS...
40346,penfold wants new visa category to attract mig...,epfnold wants new vais category to attract mgi...
90677,six legged lamb born in belgium,six legged mlab born in belgium


In [66]:
def word_by_word_accuracy(corrector, test_df):
    """Evaluate the spell corrector on a word-by-word basis."""
    total_words = 0
    correct_matches = 0
    
    for _, row in tqdm(test_df.iterrows(), position=0, leave=False):
        original_text = row['text'].split()
        augmented_text = row['augmented_text']
        
        corrected_text = correct_text(corrector, augmented_text).split()
        
        for original_word, corrected_word in zip(original_text, corrected_text):
            total_words += 1
            if original_word == corrected_word:
                correct_matches += 1

    accuracy = correct_matches / total_words if total_words > 0 else 0
    return accuracy

In [67]:
accuracy_norvig_corrector = word_by_word_accuracy(spell_corrector, test_df)
accuracy_context_corrector = word_by_word_accuracy(word_refiner, test_df)

print(f"Context Sensetive Corrector Accuracy: {accuracy_context_corrector:.2f}")
print(f"Norvig's Corrector Accuracy: {accuracy_norvig_corrector:.2f}")

Context Sensetive Corrector Accuracy: 0.28
Norvig's Corrector Accuracy: 0.20


## Evaluation on auhmented test set with a little noise 

In [25]:
test_df2 = pd.read_csv("chatgpt_prompts.csv")
test_df2 = test_df2.prompt
test_df2.head()

0    I want you to act as my personal chef. I will ...
1    I want you to act like a php interpreter. I wi...
2    I want you act as a proofreader. I will provid...
3    [Caveat Emptor: After issuing this prompt you ...
4    I want you to act as a salesperson. Try to mar...
Name: prompt, dtype: object

In [34]:
import random

def introduce_typos(text, noise_level=0.1):
    typo_text = []
    for word in text.split():
        if random.random() < noise_level:
            # Introduce a simple typo: Swap two adjacent characters
            if len(word) > 1:
                pos = random.randint(0, len(word) - 2)
                word = word[:pos] + word[pos+1] + word[pos] + word[pos+2:]
        typo_text.append(word)
    return ' '.join(typo_text)

def evaluate_accuracy(corrector, original_texts):
    correct_count = 0
    total_words = 0
    for original_text in tqdm(original_texts, desc=f"Evaluating"):
        typo_text = introduce_typos(original_text, noise_level=0.2) 
        corrected_text = correct_text(corrector, typo_text)
        original_words = original_text.split()
        corrected_words = corrected_text.split()
        
        total_words += len(original_words)
        correct_count += sum(1 for orig, corr in zip(original_words, corrected_words) if orig == corr)
    
    accuracy = correct_count / total_words if total_words > 0 else 0
    return accuracy

In [35]:
accuracy_context_corrector = evaluate_accuracy(word_refiner, test_df2)
accuracy_norvig_corrector = evaluate_accuracy(spell_corrector, test_df2)

Evaluating: 100%|██████████| 153/153 [23:51<00:00,  9.36s/it]


In [36]:
print(f"Context Sensetive Corrector Accuracy: {accuracy_context_corrector:.2f}")
print(f"Norvig's Corrector Accuracy: {accuracy_norvig_corrector:.2f}")

Context Sensetive Corrector Accuracy: 0.51
Norvig's Corrector Accuracy: 0.11


Here are the results of the evaluation:

#### Evaluation on highly augmented test set:
- Context Sensitive Corrector Accuracy: 28%
- Norvig's Corrector Accuracy: 20%

#### Evaluation on augmented test set with a little noise:
- Context Sensitive Corrector Accuracy: 51%
- Norvig's Corrector Accuracy: 11%

From the results, it's clear that the Context Sensitive Corrector performs better than Norvig's Corrector in both test sets. The overall accuracy is higher for Context Sensitive Corrector, especially on the test set with a little noise. The Context Sensitive Corrector performs better than Norvig's Corrector for several reasons:

1. **Contextual Awareness**: The Context Sensitive Corrector takes into account the context of the surrounding words in a sentence. Instead of suggesting corrections based solely on their similarity to the incorrect word, the corrector considers how well the suggestions fit within the sentence's broader context.

2. **Dynamic Data Utilization**: The Context Sensitive Corrector dynamically generates 3 and 4-grams from a 5-gram dataset.

3. **Enhanced Decision Making**: The Context Sensitive Corrector evaluates potential corrections based on their context length and frequency, ensuring that suggested corrections are not only probable but also contextually coherent. 

4. **Flexible Correction Approach**: The Context Sensitive Corrector offers correction suggestions for both misspelled and known words based on edit distance and context. Instead of accepting a known word as correct without considering the context, the Context Sensitive Corrector system takes both factors into account. 

